In [4]:
# The Oracle 
# V2.1 2024-08-29
#
import sys
import os
import json

# This line imports the print_help function from a module named common_utils, which is located within a package called utils. 
# The print_help function is likely used to display help messages to the user.
from utils.common_utils import print_help
# This line is importing several functions from the ioc_functions module, which is part of the ioc_processing package. 
# These functions are probably related to the processing of Indicators of Compromise (IOCs) in cybersecurity
from ioc_processing.ioc_functions import (
    #validate_iocs,
    parse_bulk_iocs,
    #bulk_analysis,
    process_individual_ioc_file,
    analysis,
    extract_last_analysis_date,
    format_date,
    calculate_total_malicious_score,
    classify_ioc
)
#from ioc_processing.chinmay_bulkanalysis import bulk_analysis
from file_operations.file_utils import (
    read_file,
    clean_input
)
# from ioc_processing.final_verdict import (
#     load_report,
#     save_report,
#     is_recent,
#     analyze_ioc_reports,
#     append_verdicts_to_report
# )

In [ ]:

def main():
    script_directory = "/home/jovyan/The Oracle"
    input_directory = os.path.join(script_directory, 'input_files')
    output_directory = os.path.join(script_directory, 'output_files')
    show_help = False

    if len(sys.argv) > 1 and sys.argv[1] in ['-h', '--help']:
        show_help = True

    while True:
        if show_help:
            print_help()
            show_help = False
            continue

        print("\nPlease choose your input source:")
        print("1. Input raw IOC data")
        print("2. Use IOC data from a file")
        print("3. Exit")

        choice = input("Enter choice: ").strip()

        if choice == '3':
            sys.exit("\nExiting script.")

        output_to_file = input("Do you want to save the output to a file? (yes/no): ").strip().lower() == 'yes'
        output_file_path = None

        if output_to_file:
            output_file_name = input('Please provide the name of the output file for IOC results: ')
            output_file_path = os.path.join(output_directory, output_file_name)

        aggregated_report = ""

        # Handle IOC classification and validation based on user's choice
        iocs = []
        if choice == '1':
            content = input('Please input the IOC data, separated by new lines: ')
            cleaned_content = clean_input(content)
            iocs = cleaned_content.splitlines()

        elif choice == '2':
            file_name = input('Please provide the name of the txt file for IOC input: ')
            file_path = os.path.join(input_directory, file_name)

            # Ensure file exists
            if not os.path.isfile(file_path):
                print(f"Error: File '{file_path}' not found.")
                continue

            # Retrieve the dictionary returned by process_individual_ioc_file
            ioc_dict = process_individual_ioc_file(file_path, 'auto')  # Automatically detect IOC types
            if 'auto' in ioc_dict:
                iocs = ioc_dict['auto']  # Extract the list of IOCs from the 'auto' key
            else:
                print(f"Error: Could not extract IOCs from file '{file_name}'.")
                continue

        else:
            print("Invalid input. Please choose option '1', '2', or '3'.")
            continue

        # Classify IOCs (same logic as used in the UI)
        ioc_dict = {'ips': [], 'urls': [], 'domains': [], 'hashes': []}
        for ioc in iocs:
            ioc_type = classify_ioc(ioc)
            if ioc_type != 'unknown':
                ioc_dict[f'{ioc_type}s'].append(ioc)

        # Ensure that at least one valid IOC was found
        if not any(ioc_dict.values()):
            print("Error: No valid IOCs found.")
            continue

        # Pass only non-empty categories to the analysis function
        selected_category = {k: v for k, v in ioc_dict.items() if v}
        print(f"DEBUG: Starting analysis with {selected_category}")
        aggregated_report = analysis(selected_category, output_file_path)

        # Output handling
        if output_to_file and output_file_path:
            with open(output_file_path, "a") as outfile:
                outfile.write(aggregated_report)
                outfile.write("\n")
        else:
            print(aggregated_report)

# Entry point of the script.
if __name__ == "__main__":
    main()